In [1]:
import streamlit
import numpy
from PIL import Image
import tensorflow as tf
import scipy
import cv2
import streamlit_webrtc

print(f"streamlit version: {streamlit.__version__}")
print(f"numpy version: {numpy.__version__}")
print(f"Pillow version: {Image.__version__}")
print(f"tensorflow version: {tf.__version__}")
print(f"scipy version: {scipy.__version__}")
print(f"opencv-python version: {cv2.__version__}")
print(f"streamlit-webrtc version: {streamlit_webrtc.__version__}")


streamlit version: 1.37.1
numpy version: 1.23.5
Pillow version: 10.4.0
tensorflow version: 2.17.0
scipy version: 1.13.1
opencv-python version: 4.10.0
streamlit-webrtc version: 0.47.7


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import json

# Paths to the dataset directories
train_dir = 'train'
valid_dir = 'valid'
test_dir = 'test'

# Image dimensions
img_height, img_width = 640, 640
batch_size = 16



# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True

)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'

)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'

)

# Number of classes
num_classes = len(train_generator.class_indices)

# Load MobileNetV2 model with pre-trained weights and without top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers
for layer in base_model.layers:

    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=valid_generator

)

# Save the model
model.save('staff_mobilenet_v2_model.h5')

# Save class names
class_names = list(train_generator.class_indices.keys())

with open('class_names.json', 'w') as f:

    json.dump(class_names, f)

print("Model and class names saved successfully.")

Found 63 images belonging to 3 classes.
Found 18 images belonging to 3 classes.


C:\Users\yogka\AppData\Local\Temp\ipykernel_2088\921298082.py:51: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))


Epoch 1/10


c:\Users\yogka\anaconda3\envs\mp\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 12s 2s/step - accuracy: 0.3070 - loss: 1.1130 - val_accuracy: 0.8889 - val_loss: 0.8486
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 1.0000 - loss: 0.8086 - val_accuracy: 1.0000 - val_loss: 0.6189
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 1.0000 - loss: 0.5797 - val_accuracy: 1.0000 - val_loss: 0.4403
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 1.0000 - loss: 0.4066 - val_accuracy: 1.0000 - val_loss: 0.3217
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 1.0000 - loss: 0.2970 - val_accuracy: 1.0000 - val_loss: 0.2369
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 1.0000 - loss: 0.2178 - val_accuracy: 1.0000 - val_loss: 0.1781
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 1.0000 - loss: 0.1756 - val_accuracy: 1.0000 - val_loss: 0.1380
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 1.0000 - loss: 0.1400 - val_accuracy: 1.0000 - val_loss: 0.1090
Epoch 9/10
4/4 ━━━━━━━━━━━

Model and class names saved successfully.
